In [1]:
import teradata
import pandas as pd
import matplotlib.pyplot as plt
host,user,pwd ='10.132.4.11','tsatish', "Rock@122333"
udaExec =teradata.UdaExec (appName="HelloWorld", version="1.0",logConsole=False)
with udaExec.connect(method="ODBC", system=host,username=user,password=pwd) as session:
    query1 = '''SELECT * from DD_RO_ADHOC.MERCHANT_RECOM_FINAL5'''#  sample 1000000
    MERCHANT = pd.read_sql(query1,session) 
MERCHANT.shape 

(1328565, 2)

In [4]:
MERCHANTS=MERCHANT [['PARTY_NUMR','MERCHANT_NAME']]
MERCHANTS

,PARTY_NUMR,MERCHANT_NAME
0,11748709,LOH GUAN LYE SPECIALISTS PENANG
1,1511908,AGODA INTERNATION
2,367199,GAMA S
3,12875900,COSWAY
4,1382291,HONG KONG SHENG KEE DESS SEPANG
5,3922907,KITSCHEN
6,2062801,NIRWANA MAJU
7,60398,NVCKL SBMC
8,1172913,MPY
9,3742623,COWAY


In [5]:
MERCHANTS['MERCHANT_NAME'].value_counts().head(25)

PARKSON                                   22791
AGODA HOTEL RESERVATIO                    18593
GHL                                       15816
PADINI CONCEPT STORE                      15589
WECHAT PA                                 12097
COWAY                                     11877
TMB                                       11818
MYDIN                                     11601
MOLPAY                                    11505
COTTON ON                                 10605
SECRET RECIPE                              9612
PADINI CONCEPT STR                         9404
BRANDS OUTLET                              9386
IKEA DAMANSARA                             8398
VILLAGE GROCER                             8360
CUCKOO INTERNATIONAL                       8051
ZARA                                       7715
AMWAY                                      7674
THE STORE                                  7311
IKEA                                       7223
BAR                                     

In [6]:
Mer=[]
for i in MERCHANTS['MERCHANT_NAME']:
    Mer.append(i.rstrip())
Mer
MERCHANTS['MERCHANT_NAME']=Mer
MERCHANTS
df = MERCHANTS.groupby('PARTY_NUMR').agg({'MERCHANT_NAME': ', '.join}).reset_index()
df['MERCHANT_NAME']

C:\Users\tvimal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0         COFFEE BEAN, PRUBSN GT, 0, VILLAGE GROCER, MPH...
1         JAYA GROCER SUBANG EMPIR SUBANG JAYA, JAYA GRO...
2                    JAYA GROCER MUTIARA TRPC PETALING JAYA
3         JAKEL TRADING, PRUBSN GT, STARBUCKS, TAKAFUL M...
4                             MPH BOOKSTORES, ICHIBAN RAMEN
5                                                     MYDIN
6                                        MAKNA DIRECT DEBIT
7                                        CITY CHAIN, ERAMAN
8                     ERAMAN RETAIL, JAYA GROCER SUBANG EMP
9                                                  THE BARN
10                                                HILTON KL
11                                                METROJAYA
12                             LEA CENTRE, METROJAYA, SESCO
13                    AGODA HOTEL RESERVATIO, ERAMAN, AMWAY
14                  METROJAYA, PADINI CONCEPT STORE, COSWAY
15        CHOICE PREMIER SUPERMA, UPWELL SUPERSTORE, EVE...
16                                      

In [7]:
lst=list(df.MERCHANT_NAME)
lst
list_of_list=[s.split(',') for s in lst]
list_of_list

[['COFFEE BEAN',
  ' PRUBSN GT',
  ' 0',
  ' VILLAGE GROCER',
  ' MPH BOOKSTORE',
  ' JAYA GROCER IPOH PARADE IPOH'],
 ['JAYA GROCER SUBANG EMPIR SUBANG JAYA',
  ' JAYA GROCER PEARL POINT KUALA LUMPUR',
  ' PETSWONDERLAND',
  ' MNG',
  ' KNL MARKET',
  ' JAYA GROCER SUBANG EMP'],
 ['JAYA GROCER MUTIARA TRPC PETALING JAYA'],
 ['JAKEL TRADING',
  ' PRUBSN GT',
  ' STARBUCKS',
  ' TAKAFUL MALAYSIA',
  ' H & M-SUNWAY VELOCITY',
  ' SEPHORA',
  ' TANGS',
  ' 0',
  ' VICTORIA',
  ' Traveloka',
  ' PARKSON'],
 ['MPH BOOKSTORES', ' ICHIBAN RAMEN'],
 ['MYDIN'],
 ['MAKNA DIRECT DEBIT'],
 ['CITY CHAIN', ' ERAMAN'],
 ['ERAMAN RETAIL', ' JAYA GROCER SUBANG EMP'],
 ['THE BARN'],
 ['HILTON KL'],
 ['METROJAYA'],
 ['LEA CENTRE', ' METROJAYA', ' SESCO'],
 ['AGODA HOTEL RESERVATIO', ' ERAMAN', ' AMWAY'],
 ['METROJAYA', ' PADINI CONCEPT STORE', ' COSWAY'],
 ['CHOICE PREMIER SUPERMA',
  ' UPWELL SUPERSTORE',
  ' EVERRISE',
  ' PTPTN',
  ' NANDO',
  ' EVERRISE D',
  ' METROJAYA',
  ' SERVAY H',
  ' GIANT'],

In [8]:
from apyori import apriori
association_rules = apriori(list_of_list, min_support=0.001, min_confidence=0.05, min_lift=1, min_length=2)  
association_results = list(association_rules)
association_results

[RelationRecord(items=frozenset({' AGODA HOTEL RESERVATIO', ' BRANDS OUTLET'}), support=0.0011351417315401317, ordered_statistics=[OrderedStatistic(items_base=frozenset({' BRANDS OUTLET'}), items_add=frozenset({' AGODA HOTEL RESERVATIO'}), confidence=0.07213930348258707, lift=2.6091442769496953)]),
 RelationRecord(items=frozenset({' FAVE', ' AGODA HOTEL RESERVATIO'}), support=0.0011420492877158323, ordered_statistics=[OrderedStatistic(items_base=frozenset({' FAVE'}), items_add=frozenset({' AGODA HOTEL RESERVATIO'}), confidence=0.09868682849184242, lift=3.569318822602149)]),
 RelationRecord(items=frozenset({' GHL', ' AGODA HOTEL RESERVATIO'}), support=0.002350871618463437, ordered_statistics=[OrderedStatistic(items_base=frozenset({' AGODA HOTEL RESERVATIO'}), items_add=frozenset({' GHL'}), confidence=0.08502664890073285, lift=3.1323834764721843), OrderedStatistic(items_base=frozenset({' GHL'}), items_add=frozenset({' AGODA HOTEL RESERVATIO'}), confidence=0.08660615828314532, lift=3.1323

In [9]:
association_results
Result = pd.DataFrame(columns=('Items','Support','Confidence','Lift'))#'Antecedent','Consequent',

Support =[]
Confidence = []
Lift = []
Items = []
#Antecedent = []
#Consequent=[]

for RelationRecord in association_results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
       # Antecedent.append(ordered_stat.items_base)
       #Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)

Result['Items'] =Items# list(map(set, Items))                                   
#df['Antecedent'] = list(map(set, Antecedent))
#df['Consequent'] = list(map(set, Consequent))
Result['Support'] = Support
Result['Confidence'] = Confidence
Result['Lift']= Lift
Result
Result.sort_values(by ='Lift', ascending = False, inplace = True)
print(df.shape)
Result['Items']=[list(x) for x in list(Result['Items'])]
Result

(434307, 2)


,Items,Support,Confidence,Lift
16,"[ BIGPA, BIGPAY]",0.001669,0.217652,67.471764
17,"[ BIGPA, BIGPAY]",0.001669,0.517488,67.471764
100,"[ TNG, TNG WALLET E]",0.001209,0.174535,22.742751
99,"[ TNG, TNG WALLET E]",0.001209,0.157516,22.742751
19,"[ BIGPA, BOOST APP]",0.001103,0.116121,15.140275
18,"[ BIGPA, BOOST APP]",0.001103,0.143801,15.140275
28,"[ CARING PHARMACY, CARING]",0.001004,0.125793,12.839709
27,"[ CARING PHARMACY, CARING]",0.001004,0.102468,12.839709
78,"[ NANDO, STARBUCKS]",0.001547,0.125607,11.530798
77,"[ NANDO, STARBUCKS]",0.001547,0.142042,11.530798


In [10]:
df2=Result[Result.Lift<100]
df2#.to_excel('C:\\Users\\tvimal\\Desktop\\Recomended.xlsx')

,Items,Support,Confidence,Lift
16,"[ BIGPA, BIGPAY]",0.001669,0.217652,67.471764
17,"[ BIGPA, BIGPAY]",0.001669,0.517488,67.471764
100,"[ TNG, TNG WALLET E]",0.001209,0.174535,22.742751
99,"[ TNG, TNG WALLET E]",0.001209,0.157516,22.742751
19,"[ BIGPA, BOOST APP]",0.001103,0.116121,15.140275
18,"[ BIGPA, BOOST APP]",0.001103,0.143801,15.140275
28,"[ CARING PHARMACY, CARING]",0.001004,0.125793,12.839709
27,"[ CARING PHARMACY, CARING]",0.001004,0.102468,12.839709
78,"[ NANDO, STARBUCKS]",0.001547,0.125607,11.530798
77,"[ NANDO, STARBUCKS]",0.001547,0.142042,11.530798


In [ ]:
df.to_excel('C:\\Users\\tvimal\\Desktop\\Recomended_new.xlsx')

In [11]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

TE = TransactionEncoder()
TE_ary = TE.fit(list_of_list).transform(list_of_list)
dfd = pd.DataFrame(TE_ary, columns=TE.columns_)
dfd.head()

,*MYR-ANYTIME-EZYPAYV2,*THE CHINA PRESS-1,0,11STREET,2C2P*ZARA COM,2C2P-CHILINDO,2C2P-JOMPARKING,2C2P-TRAVEL RECOMMEND,A HONG,AA PHARMACY,...,YTL COMMS,YUBISO,YUNG FA RETAIL,ZALORA MALAYSIA,ZARA,ZENXIN ORGANIC FOOD,ZENZU,Zalora SEA,doTERRA Malaysia,uCloudlink
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
from mlxtend.frequent_patterns import apriori
apriori(dfd, min_support=0.01, use_colnames=True)

,support,itemsets
0,0.027649,( AGODA HOTEL RESERVATIO)
1,0.015735,( BRANDS OUTLET)
2,0.016574,( COTTON ON)
3,0.011572,( FAVE)
4,0.027144,( GHL)
5,0.011197,( IKEA)
6,0.013373,( IKEA DAMANSARA)
7,0.016198,( MOLPAY)
8,0.012307,( MYDIN)
9,0.010893,( NANDO)
